In [107]:
import pandas as pd
import requests
from io import BytesIO
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Image
import matplotlib.pyplot as plt

In [108]:
df = pd.read_excel ("LH_20231002.xlsx")

In [109]:
#set extracted single urls of FOTO/VIDEO as Link_foto
Link_foto = df['FOTO/VIDEO PEKERJAAN']

#split C
df = df.assign(Link_foto=Link_foto.str.split('; ')).explode('Link_foto', ignore_index=True)

In [110]:
#pick column names
primary_columns = ['ID', 'TANGGAL LAPORAN', 'Name', 'ITEM PEKERJAAN', 'QUANTITY PEKERJAAN', 'SATUAN', 'JUMLAH TENAGA KERJA', 'PERALATAN (JIKA ADA)','MATERIAL (JIKA ADA)', 'DESKRIPSI / KENDALA', 'Link_foto']

#new primary DF
primary_df = df[primary_columns]

In [111]:
#import PILLOW
from PIL import Image as PILImage

In [112]:
#testing with 4 tail
df = primary_df
test_df = df.tail(4)

#empty list to store image
images = []

In [104]:
#Initialize the 'img' variable as none outside of the loop
img = None

#fetch images from URLS, resize and store in images list
for url in test_df['Link_foto']:
    response = requests.get(url)
    if response.status_code == 200:
        img_data = BytesIO(response.content)
        img = PILImage.open(img_data)

        #check if img is not None before resizing
        if img:
            #resize the images while preserving aspect ratio 200px width
            img.thumbnail(200, 200)

            reportlab_image = Image(img, width=200, height=200)
            images.append(reportlab_image)
        else:
            images.append(None)
    else:
        images.append(None)

In [117]:
# Iterate through images and display them
for i, image in enumerate(images):
    if image:
        plt.imshow(image, cmap='gray')
        plt.title(f'Image {i + 1}')
        plt.show()

In [114]:
print(plt.show)

<function show at 0x000001A8050DC400>


In [ ]:
#create a pdf document
pdf_filename = "test_output.pdf"
doc = SimpleDocTemplate(pdf_filename, pagesize=letter)

#define the layout : 10 rows per page
rows_per_page = 10
page_width, page_height = letter
image_width = page_width

In [93]:
df = primary_df
images = []

#Fetch Images from URL
for url in df['Link_foto']:
    response = requests.get(url)
    if response.status_code == 200:
        img_data = BytesIO(response.content)
        img = Image(img_data)
        images.append(img)
    else:
        images.append(None)

KeyboardInterrupt: 

In [88]:
print(primary_df)

      ID TANGGAL LAPORAN                    Name  \
0      3      2023-07-11        Mulyadi Prasetyo   
1      5      2023-07-12        Mulyadi Prasetyo   
2      6      2023-07-13  Muhammad Faris Eryando   
3      6      2023-07-13  Muhammad Faris Eryando   
4      6      2023-07-13  Muhammad Faris Eryando   
..   ...             ...                     ...   
378  132      2023-09-25        Mulyadi Prasetyo   
379  132      2023-09-25        Mulyadi Prasetyo   
380  133      2023-09-26        Mulyadi Prasetyo   
381  133      2023-09-26        Mulyadi Prasetyo   
382  133      2023-09-26        Mulyadi Prasetyo   

                                        ITEM PEKERJAAN  QUANTITY PEKERJAAN  \
0                                  Jalur plumbing lama                   2   
1                                 Galian pipa plumbing                   3   
2    Titik pondasi progreaa dan dinding calon klose...                   1   
3    Titik pondasi progreaa dan dinding calon klose...         

In [82]:
#export to excel
#xlsx path
excel_file_path = "PrimaryState_LH.xlsx"

#write to xlsx
primary_df.to_excel(excel_file_path, index=False)